In [2]:
import collections
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval
# !pip install transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import time
import torch.nn.utils.prune as prune
import numpy as np
import os

In [3]:
PATH_TO_EMBEDS = '/home/aayush/Documents/pclub_secy/task_4/compressed_array.npz'
PATH_TO_DF = '/home/aayush/Documents/pclub_secy/task_4/compressed_dataframe.csv.gz'

In [4]:
def run_faiss_search(query_text, top_k):
    query = [query_text]
    query_embedding = model.encode(query)
    scores, index_vals = faiss_index.search(query_embedding, top_k)
    index_vals_list = index_vals[0]
    
    return index_vals_list
    

def run_rerank(index_vals_list, query_text):  
    chunk_list = list(df_data['paragraph'])
    pred_strings_list = [chunk_list[item] for item in index_vals_list]
    cross_input_list = []
    for item in pred_strings_list:
        new_list = [query_text, item]
        cross_input_list.append(new_list)
    df = pd.DataFrame(cross_input_list, columns=['query_text', 'pred_text'])
    df['original_index'] = index_vals_list
    cross_scores = cross_encoder.predict(cross_input_list)
    df['cross_scores'] = cross_scores
    df_sorted = df.sort_values(by='cross_scores', ascending=False)
    df_sorted = df_sorted.reset_index(drop=True)

    pred_list = []

    for i in range(0,len(df_sorted)):
        text = df_sorted.loc[i, 'pred_text']
        original_index = df_sorted.loc[i, 'original_index']
        item = {
            'abstract': text
        }
        pred_list.append(item)
    return pred_list


def print_search_results(pred_list, num_results_to_print):
    for i in range(0,num_results_to_print):      
        pred_dict = pred_list[i]
        para = pred_dict['abstract']
        print('Abstract:',para)
        print()
    
   
def run_search(query_text, num_results_to_print, top_k=5):
    pred_index_list = run_faiss_search(query_text, top_k)
    pred_list = run_rerank(pred_index_list, query_text)
    return pred_list

In [5]:
import torch
torch.cuda.empty_cache()
device = 0 if torch.cuda.is_available() else -1


In [6]:
model = AutoModelForQuestionAnswering.from_pretrained("navteca/electra-base-squad2")
tokenizer = AutoTokenizer.from_pretrained('navteca/electra-base-squad2')

config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [7]:
nlp = pipeline("question-answering", model = model, tokenizer= tokenizer,device=device)


In [8]:
modules = [module for module in nlp.model.modules()]
mods = []
def get_modules(mod,parent):
    mod_list = list(mod._modules.keys())
    if(len(mod_list)>0):    
        for mod_n in mod_list:
#             print(mod_n)
            if(mod_n in ['qa_outputs']):
                continue
            get_modules(mod._modules[mod_n],mod)
    else:
        mods.append((str(type(mod)).split('.')[-1][:-2],mod))
        return

In [9]:
get_modules(modules[0],None)
final = [(name,layer) for name,layer in mods if name not in ['Dropout','GELUActivation','LayerNorm','Softmax','Tanh','MatMulWrapper', 'SqueezeBertLayerNorm']]

unc = ['Linear']
for name,layer in final:
    if name not in unc:
        unc.append(name)
unc

['Linear', 'Embedding']

In [10]:
parameters_to_prune = [
    (module, "weight") for name,module in final
]
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured, 
    amount=0.3,
)
for j in range(len(final)):
    if prune.is_pruned(final[j][1]):
        prune.remove(final[j][1],'weight')

In [11]:
torch.save(model,'pruned.pt')

In [12]:
from torch import nn
from torch.quantization import quantize_dynamic
import torch
model = AutoModelForQuestionAnswering.from_pretrained("navteca/electra-base-squad2")
model_q = quantize_dynamic(
    model=model, qconfig_spec={nn.Linear}, dtype=torch.qint8, inplace=False
)
     
nlp = pipeline("question-answering", model = model_q, tokenizer= "navteca/electra-base-squad2")
torch.save(model_q,'quantized.pt')


In [13]:
pip install git+https://github.com/huggingface/optimum-intel.git


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Cloning https://github.com/huggingface/optimum-intel.git to /tmp/pip-req-build-7bfb70jr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/optimum-intel.git /tmp/pip-req-build-7bfb70jr
  Resolved https://github.com/huggingface/optimum-intel.git to commit 1319d7bec80622abdb39b7d0307df6e453e4e903
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 10.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.3 MB/s eta 0:00:00
  Created wheel for optimum-intel: filename=optimum_intel-1.17.0.dev0+1319d7b-py3-none-any.whl size=215281 sha256=2ad5e743004bcde9811bcc09a1304e1892a635eb7463e60f8e07a98125f15c1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-jlg9x5i6/wheels/99/e8/f9/680e64aac521c117548ed9511e82474364af

In [14]:
!python -m pip install optimum[neural-compressor]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 67.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 21.0 MB/s eta 0:00:00


In [15]:
from transformers import AutoModelForQuestionAnswering
from neural_compressor.config import PostTrainingQuantConfig
from optimum.intel.neural_compressor import INCQuantizer

model = AutoModelForQuestionAnswering.from_pretrained("navteca/electra-base-squad2")
# The directory where the quantized model will be saved
save_dir = "quantized_model"
# Load the quantization configuration detailing the quantization we wish to apply
quantization_config = PostTrainingQuantConfig(approach="dynamic")
quantizer = INCQuantizer.from_pretrained(model)
# Apply dynamic quantization and save the resulting model
quantizer.quantize(quantization_config=quantization_config, save_directory=save_dir)

2024-05-23 18:23:16 [INFO] Start auto tuning.
2024-05-23 18:23:16 [INFO] Execute the tuning process due to detect the evaluation function.
2024-05-23 18:23:16 [INFO] Adaptor has 5 recipes.
2024-05-23 18:23:16 [INFO] 0 recipes specified by user.
2024-05-23 18:23:16 [INFO] 3 recipes require future tuning.
2024-05-23 18:23:16 [INFO] *** Initialize auto tuning
2024-05-23 18:23:16 [INFO] {
2024-05-23 18:23:16 [INFO]     'PostTrainingQuantConfig': {
2024-05-23 18:23:16 [INFO]         'AccuracyCriterion': {
2024-05-23 18:23:16 [INFO]             'criterion': 'relative',
2024-05-23 18:23:16 [INFO]             'higher_is_better': True,
2024-05-23 18:23:16 [INFO]             'tolerable_loss': 0.01,
2024-05-23 18:23:16 [INFO]             'absolute': None,
2024-05-23 18:23:16 [INFO]             'keys': <bound method AccuracyCriterion.keys of <neural_compressor.config.AccuracyCriterion object at 0x7d67bb7170d0>>,
2024-05-23 18:23:16 [INFO]             'relative': 0.01
2024-05-23 18:23:16 [INFO]    

In [16]:
def pred_theme_ans(questions, pred_out,threshold=0.05):
  theme = questions[0]["Theme"]
  for question in questions:
    ans = {}
    ans["question_id"] = question["id"]
    QA_input = {
      'question':question['Question'],
      'context': question['Paragraph']
      }
    res = nlp(QA_input) 
    score = res['score']

    if (score<threshold):
       ans["answers"]= ''
    else:
      ans["answers"]=res['answer']
    pred_out.append(ans)

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(str(s)))))
def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()
def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

# Calculate the f1 score between prediction and multiple ground truths
def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  try:
    ground_truths[0]
  except Exception as e:
    if predicted!=predicted:
      max_f1 = 1
  return max_f1

In [17]:
from transformers import AutoTokenizer, pipeline
tokenizer = AutoTokenizer.from_pretrained("navteca/electra-base-squad2")

from optimum.intel.neural_compressor import INCModelForQuestionAnswering

loaded_model_from_hub = INCModelForQuestionAnswering.from_pretrained(
    "quantized_model"
)

nlp = pipeline("question-answering", model = loaded_model_from_hub, tokenizer= tokenizer)

loading configuration file quantized_model/inc_config.json
INCConfig {
  "distillation": {},
  "neural_compressor_version": "2.5.1",
  "optimum_version": "1.19.2",
  "pruning": {},
  "quantization": {
    "dataset_num_samples": null,
    "is_static": false
  },
  "save_onnx_model": false,
  "torch_version": "2.1.2+cpu",
  "transformers_version": "4.39.3"
}

Quantized model was obtained with torch version 2.1.2+cpu but 2.1.2+cpu was found.


In [18]:
# from tqdm import tqdm
# # questions = json.loads(questions_df.to_json(orient="records"))
# # theme_intervals = json.loads(theme_df.to_json(orient="records"))
# pred_out = []
# theme_inf_time = {}
# execution_times = []
# for theme_interval in tqdm(theme_intervals):
#   theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
#   execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, pred_out), number=1)
#   execution_times.append(execution_time/len(theme_ques))
#   theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
# pred_df = pd.DataFrame.from_records(pred_out)
# # Write prediction to a CSV file. Teams are required to submit this csv file.
# pred_df.to_csv('/content/output_prediction.csv', index=False)
# print("\navg_inference_time:",round(sum(execution_times)/len(theme_intervals),3)*1000)

In [19]:
def pred_theme_ans_sing(question, para,threshold=0.001):
    
#   theme = questions[0]["Theme"]
#   for question in questions:
#     ans = {}
#     # Get Question Id
#     ans["question_id"] = question["id"]
    QA_input = {
      'question':question,
      'context': para
      }
    
    # Get Result, Contains, Answer Start, Answer End, Answer and the Confidence Score
    res = nlp(QA_input) 
    score = res['score']
    print(score)
    # If Confidence Score is less than the threshold then dont Answer,otherwise answer
    if (score<threshold):
       ans= ''
    else:
      ans=res['answer']
    print(ans)

In [20]:
q = "What were the names given to the various dialects predating the 19th century?"
para = "Throughout the history of the South Slavs, the vernacular, literary, and written languages (e.g. Chakavian, Kajkavian, Shtokavian) of the various regions and ethnicities developed and diverged independently. Prior to the 19th century, they were collectively called \"Illyric\", \"Slavic\", \"Slavonian\", \"Bosnian\", \"Dalmatian\", \"Serbian\" or \"Croatian\". As such, the term Serbo-Croatian was first used by Jacob Grimm in 1824, popularized by the Vienna philologist Jernej Kopitar in the following decades, and accepted by Croatian Zagreb grammarians in 1854 and 1859. At that time, Serb and Croat lands were still part of the Ottoman and Austrian Empires. Officially, the language was called variously Serbo-Croat, Croato-Serbian, Serbian and Croatian, Croatian and Serbian, Serbian or Croatian, Croatian or Serbian. Unofficially, Serbs and Croats typically called the language \"Serbian\" or \"Croatian\", respectively, without implying a distinction between the two, and again in independent Bosnia and Herzegovina, \"Bosnian\", \"Croatian\", and \"Serbian\" were considered to be three names of a single official language. Croatian linguist Dalibor Brozović advocated the term Serbo-Croatian as late as 1988, claiming that in an analogy with Indo-European, Serbo-Croatian does not only name the two components of the same language, but simply charts the limits of the region in which it is spoken and includes everything between the limits (‘Bosnian’ and ‘Montenegrin’). Today, use of the term \"Serbo-Croatian\" is controversial due to the prejudice that nation and language must match. It is still used for lack of a succinct alternative, though alternative names have been used, such as Bosnian/Croatian/Serbian (BCS), which is often seen in political contexts such as the Hague War Crimes tribunal."

In [21]:
pred_theme_ans_sing(q,para)

0.006296506151556969
"Serbian" or "Croatian"


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

# Sentences are encoded by calling model.encode()
embeddings = model.encode(chunk_list)

print(embeddings.shape)
print('Embedding length', embeddings.shape[1])
ress = run_search(str(q), 5)[0]['abstract']
pred_theme_ans_sing(q,ress)